In [2]:
import multiprocessing as mp
import time
import re
import os.path
import random
import pandas as pd
from bs4 import BeautifulSoup
from seleniumwire import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl import load_workbook


start_time = time.time()

url = 'https://market.csgo.com/item/3757781165'


filename = '\parsing_cs.xlsx'



global driver


def get_html(url):

    users = [{'http_proxy': '193.31.103.210:9244',
              'user-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:94.0) Gecko/20100101 Firefox/94.0'},
             {'http_proxy': '193.31.101.224:9604',
              'user-agent': 'Opera/9.80 (J2ME/MIDP; Opera Mini/5.0.18635/886; U; en) Presto/2.4.15'},
             {'http_proxy': '193.31.102.78:9514',
              'user-agent': 'Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.209.0 Safari/532.0'}]

    persona = random.choice(users)


    # PROXY = persona['http_proxy']
    # webdriver.DesiredCapabilities.CHROME['proxy'] = {
    #     "httpProxy": PROXY,
    #     "ftpProxy": PROXY,
    #     "sslProxy": PROXY,
    #     "proxyType": "MANUAL",
    #
    # }
    #webdriver.DesiredCapabilities.CHROME['acceptSslCerts'] = True



    options = webdriver.ChromeOptions()
    options.add_argument(f"user-agent={persona['user-agent']}")
    #options.add_argument("--disable-blink-features=AutomationControlled")


    #options.add_argument(f"proxy-server={persona['http_proxy']}")

    s = Service(executable_path="C:/Users/venag/Downloads/chromedriver_win32/chromedriver.exe")

    driver = webdriver.Chrome(options=options, service=s)

    try:
        #driver.request_interceptor = lib.headers.interceptor
        driver.get(url)
        #wait = WebDriverWait(driver, 5)
        #time.sleep(5)
        #wait.until(EC.presence_of_element_located((By.CLASS_NAME, "collapsible__toggle-wrap")))
        html = driver.page_source
    except:
        html = None
    finally:
        driver.close()
        driver.quit()
    return html


if __name__ == '__main__':

    #p = mp.Pool(processes=10)
    #html = p.map(get_html, all_url)
    html = get_html(url)
    #print(html)




    print("--- %s seconds ---" % (time.time() - start_time))

--- 43.559950828552246 seconds ---


In [2]:
soup = BeautifulSoup(html, 'html.parser')

In [12]:
price_all = soup.find_all('button', attrs={'class': 'ibutton ibutton-buy'})
percent_all = soup.find_all('div', attrs={'class': 'sameitem-floatnum'})

In [23]:
data = []
def datas(i,j):
    mass = {
        'price' : i.get_text().replace('\xa0 ','').replace('\n',''),
        'percent' : j.find('span').get_text().replace('%',''),
        'url' : i['onclick']#.replace('\xa0 ','').replace('\n',''),
    }

    return mass

for i,j in zip(price_all,percent_all):
    data.append(datas(i,j))
data[0]

{'price': '94.88',
 'percent': '17.755',
 'url': "BuyItem(4578724305, 519977179, 9488, '0', 1, 3757771185)"}

In [29]:
df = pd.DataFrame(columns = ['price','percent','url'])
for i in range(len(data)):
    df.loc[i] = data[i]

df.to_excel(os.getcwd()+filename, index = False)